In [1]:
import numpy as np
import plotly.express as px
from instance_mongodb import instance_mongodb_sei

from pymatgen.core.structure import Molecule

import torch

from e3nn import o3

In [2]:
def rotate_three_dimensions(alpha, beta, gamma):
    """Rotate the molecule by arbitrary angles alpha
    beta and gamma."""
    cos = np.cos
    sin = np.sin

    r_matrix = [
        [
            cos(alpha) * cos(beta),
            cos(alpha) * sin(beta) * sin(gamma) - sin(alpha) * cos(gamma),
            cos(alpha) * sin(beta) * cos(gamma) + sin(alpha) * sin(gamma),
        ],
        [
            sin(alpha) * cos(beta),
            sin(alpha) * sin(beta) * sin(gamma) + cos(alpha) * cos(gamma),
            sin(alpha) * sin(beta) * cos(gamma) - cos(alpha) * sin(gamma),
        ],
        [-sin(beta), cos(beta) * sin(gamma), cos(beta) * cos(gamma)],
    ]

    r_matrix = np.array(r_matrix)

    return r_matrix

In [14]:
# Visualise computed Hamiltonian
db = instance_mongodb_sei(project="mlts")
collection = db.rotated_waters_dataset

hamiltonians = []
angles = []
structures = []
overlap_matrices = []

for doc in collection.find({}):
    hamiltonians.append(doc["fock_matrices"][0])
    overlap_matrices.append(doc["overlap_matrices"][0])
    structure = Molecule.from_dict(doc["structures"][0])
    structures.append(structure)
    angles.append(doc["angles"])

hamiltonians = np.array(hamiltonians)
overlap_matrices = np.array(overlap_matrices)
angles = np.array(angles)

print(f"Shape of hamiltonians: {hamiltonians.shape}")

fig = px.imshow(
    hamiltonians[:, 1, ...], animation_frame=0, labels=dict(x="Basis", y="Basis", color="Value")
)

fig.show()

Shape of hamiltonians: (100, 2, 7, 7)


In [25]:
def subdiagonalize_matrix(indices, matrix_H, matrix_S):
    """Subdiagonalise the matrix."""
    sub_matrix_H = matrix_H.take(indices, axis=0).take(indices, axis=1)
    sub_matrix_S = matrix_S.take(indices, axis=0).take(indices, axis=1)

    eigenval, eigenvec = np.linalg.eig(np.linalg.solve(sub_matrix_S, sub_matrix_H))

    # Normalise the eigenvectors
    for col in eigenvec.T:
        col /= np.sqrt(np.dot(col.conj(), np.dot(sub_matrix_S, col)))
    
    t_matrix = np.identity(matrix_H.shape[0])

    for i in range(len(indices)):
        for j in range(len(indices)):
            t_matrix[indices[i],indices[j]] = eigenvec[i,j]

    # Unitary transform to get the rearranged Hamiltonian and overlap
    H_r = np.dot(np.transpose(np.conj(t_matrix)), np.dot(matrix_H, t_matrix))
    S_r = np.dot(np.transpose(np.conj(t_matrix)), np.dot(matrix_S, t_matrix))

    return H_r, S_r, eigenval


In [35]:
# Subdiagonalize the Hamiltonian

indices_all = [[0], [1, 2, 3, 4, 5], [6]]

hamiltonians_sub = []
overlap_matrices_sub = []
coupling = []

for i in range(hamiltonians.shape[0]):
    spin_up_H = hamiltonians[i, 0, ...]
    spin_up_S = overlap_matrices[i, 0, ...]
    H_r = spin_up_H
    S_r = spin_up_S
    for indices in indices_all:
        H_r, S_r, eigenval = subdiagonalize_matrix(indices, H_r, S_r) 
    hamiltonians_sub.append(H_r)
    overlap_matrices_sub.append(S_r)

    # Subtract the diagonal elements from the Hamiltonian
    H_r = H_r - np.diag(np.diag(H_r))
    coupling.append(H_r)

# Plot the subdiagonalized Hamiltonian
hamiltonians_sub = np.array(hamiltonians_sub)
overlap_matrices_sub = np.array(overlap_matrices_sub)
coupling = np.array(coupling)

fig = px.imshow(
    hamiltonians_sub[:, ...], animation_frame=0, labels=dict(x="Basis", y="Basis", color="Value")
)
fig.show()

# Plot the overlap matrix
fig = px.imshow(
    overlap_matrices_sub[:, ...], animation_frame=0, labels=dict(x="Basis", y="Basis", color="Value")
)
fig.show()

# Plot the coupling matrix
fig = px.imshow(
    coupling[:, ...], animation_frame=0, labels=dict(x="Basis", y="Basis", color="Value")
)
fig.show()


In [27]:
# For each sub-diagonalized Hamiltonian, convert the matrix to the right dimensions using a fully connected 
# tensor product
irreps_in = o3.Irreps("1x0e + 2x0e + 1x1o + 1x0e")
print(f"Dimension of irreps_in: {irreps_in.dim}")
irreps_intermediate = o3.Irreps("1x0e + 2x0e + 1x1o + 1x0e")
irreps_out = o3.Irreps("1x0e + 1x0e + ")

Dimension of irreps_in: 7


In [ ]:
irreps_hamiltonian = o3.Irreps("2x0e + 1x1o + 2x0e")
print(f"Dimension of the Hamiltonian space: {irreps_hamiltonian.dim}")

D_matrices = []
for idx, angle in enumerate(angles):
    alpha, beta, gamma = angle

    rotation_matrix = rotate_three_dimensions(alpha, beta, gamma)
    rotation_matrix = torch.tensor(rotation_matrix)
    if idx == 0:
        rotation_matrix_0 = rotation_matrix

    # Reference the rotation matrix to the first one
    rotation_matrix = rotation_matrix @ rotation_matrix_0.T

    D_matrix = irreps_hamiltonian.D_from_matrix(rotation_matrix)
    D_matrices.append(D_matrix)

D_matrices = torch.stack(D_matrices)
# Convert to numpy array
D_matrices = D_matrices.detach().numpy()

fig = px.imshow(
    D_matrices, animation_frame=0, labels=dict(x="Basis", y="Basis", color="Value")
)
fig.show()

In [ ]:
# Referencing all the angles to the first angle, compute the Hamiltonian in the rotated basis
hamiltonians_rotated = []
for i in range(len(angles)):
    hamiltonian_rotated = D_matrices[i] @ hamiltonians[0, ..., :, :] @ D_matrices[i].T
    hamiltonians_rotated.append(hamiltonian_rotated)

hamiltonians_rotated = np.array(hamiltonians_rotated)
hamiltonians_rotated_diff = hamiltonians_rotated - hamiltonians

fig = px.imshow(
    hamiltonians_rotated_diff[:, 0, ...],
    animation_frame=0,
    labels=dict(x="Basis", y="Basis", color="Value"),
    range_color=[-1, 1],
)
fig.show()